# Main Codes used for Paper 1

In [ ]:
import sys
sys.path.append("..")
import pandas as pd 
from src.arima import ArimaV2 as Arima
from src.arima import Arima as Arima2
from src.genetic import Genetic
from influxdb import InfluxDBClient
from datetime import datetime
from dateutil import relativedelta
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np
from src.intersection import intersection
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

TODAY = datetime.strptime('15/03/2020', '%d/%m/%Y')
START = TODAY - relativedelta.relativedelta(days=60)
START = int(datetime.timestamp(START)*1e9)

SAVE = True

# 1) Market Clearing
## MI Clearing

In [ ]:
client = InfluxDBClient('localhost', 8086, 'root', 'root', 'PublicBids')
res = client.query("select * from demandMI where time='2020-04-04T00:00:00Z'").raw
bid = pd.DataFrame(res['series'][0]['values'], columns = res['series'][0]['columns']).set_index('op').drop(columns=['time'])
res = client.query("select * from supplyMI where time='2020-04-04T00:00:00Z'").raw
off = pd.DataFrame(res['series'][0]['values'], columns = res['series'][0]['columns']).set_index('op').drop(columns=['time'])
bid.Price = bid.Price.replace(0, 3000)

sup = off[(off >= 0).any(1)]
dem = bid[(bid >= 0).any(1)]
# Sort the prices
sup = off.sort_values('Price', ascending=True)
dem = bid.sort_values('Price', ascending=False)
# Cumulative sums of quantity
sup_cum = np.cumsum(sup['Quantity'])
dem_cum = np.cumsum(dem['Quantity'])
clearing = intersection( 
    dem_cum.values, 
    dem.Price.values, 
    sup_cum.values, 
    sup.Price.values,
)

rcParams.update({'figure.autolayout': True})
rcParams.update({'font.size': 12})

plt.figure(figsize=(7.4,4.8))
plt.plot(dem_cum, dem.Price.values, linewidth = 2, label='Supply', color='steelblue')
plt.plot(sup_cum, sup.Price.values, linewidth = 2, label='Demand', color='salmon', linestyle='-.')
plt.axhline(y=clearing[1][0], linestyle= (0, (5, 10)), color='k', label='Clearing Price')
plt.ylim(0, 300)
plt.grid(linestyle='-.')
plt.legend(ncol=3)
plt.xlabel('Quantity [MWh]')
plt.ylabel('Price [\u20ac/MWh]')
if SAVE:
    plt.savefig('../fig/CurvesMI.png', transparent=True)

## MSD Supply

In [ ]:
th = client.query(f"SELECT * FROM STRes where time='2020-04-04T00:00:00Z'").raw['series'][0]['values'][0][1]

client = InfluxDBClient('localhost', 8086, 'root', 'root', 'PublicBids')
res = client.query("select * from demandMSD where time='2020-04-04T00:00:00Z'").raw
bid = pd.DataFrame(res['series'][0]['values'], columns = res['series'][0]['columns']).set_index('op').drop(columns=['time'])
res = client.query("select * from supplyMSD where time='2020-04-04T00:00:00Z'").raw
off = pd.DataFrame(res['series'][0]['values'], columns = res['series'][0]['columns']).set_index('op').drop(columns=['time'])
bid.Price = bid.Price.replace(0, 3000)

sup = off[(off >= 0).any(1)]
dem = bid[(bid >= 0).any(1)]
# Sort the prices
sup = off.sort_values('Price', ascending=True)
dem = bid.sort_values('Price', ascending=False)
# Cumulative sums of quantity
sup_cum = np.cumsum(sup['Quantity'])
dem_cum = np.cumsum(dem['Quantity'])
# Create the th curve
x_th = np.array([th, th])
y_th = np.array([0, np.max(sup.Price.values)])
clearing = intersection( 
    sup_cum.values, 
    sup.Price.values, 
    x_th,
    y_th
)

rcParams.update({'figure.autolayout': True})
rcParams.update({'font.size': 12})

plt.figure(figsize=(7.4,4.8))
plt.plot(sup_cum, sup.Price.values, linewidth = 2, label='Supply', color='steelblue')
plt.plot(x_th, y_th, linewidth = 2, label='Terna Threshold', color='salmon', linestyle='-.')
plt.axhline(y=clearing[1][0], linestyle= (0, (5, 10)), color='k', label='Clearing Price')
plt.ylim(0, 300)
plt.grid(linestyle='-.')
plt.legend()

plt.xlabel('Quantity [MWh]')
plt.ylabel('Price [\u20ac/MWh]')
if SAVE:
    plt.savefig('../fig/CurvesMSD_s.png', transparent=True)

# 2) Performances Evaluation
## Forecasting Analysis

In [ ]:
arima = pd.read_csv('../data/arima1days.csv')
lstm = pd.read_csv('../data/lstm1days.csv')
arima = arima.drop(988)
lstm = lstm.drop(np.arange(1041,1048))
lstm = lstm.drop(np.arange(0,53))

rcParams.update({'figure.autolayout': True})
rcParams.update({'font.size': 12})

plt.plot(lstm.y.values[530:561], label='Observation', marker='o', color='steelblue', markersize=4)
plt.plot(arima.y_hat.values[530:561], label='ARIMA(0,1,0)', marker='^', linestyle='--', color='brown', markersize=4)

plt.legend()
plt.xlabel('Days')
plt.ylabel('Daily Demanded Quantity [MWh]')
plt.grid('--')
plt.ylim(8500, 20000)
plt.xlim(0)
if SAVE:
    plt.savefig('../fig/arimacomparison.png', transparent=True)

In [ ]:
mse = mean_squared_error(lstm.y.values, arima.y_hat.values)
mape = np.mean(np.abs((lstm.y.values - arima.y_hat.values) / lstm.y.values)) * 100
mae = mean_absolute_error(lstm.y.values, arima.y_hat.values)
r2 = r2_score(lstm.y.values, arima.y_hat.values)
print(f'\tMSE: {round(mse,2)}')
print(f'\tMAE: {round(mae,2)}')
print(f'\tMAPE: {round(mape,2)}%')
print(f'\tR2: {round(r2,2)}')

In [ ]:
OP_LIST = []
for market in ['MGP', 'MI', 'MSD']:
    res = client.query(f"SELECT * FROM demand{market} WHERE time >= {START}").raw
    for val in res['series'][0]['values']:
        if val[3] not in OP_LIST and "'" not in val[3]:
            OP_LIST.append(val[3])

    res = client.query(f"SELECT * FROM supply{market} WHERE time >= {START}").raw
    for val in res['series'][0]['values']:
        if val[3] not in OP_LIST and "'" not in val[3]:
            OP_LIST.append(val[3])

y_hat = []
y = []

TODAY = datetime.strptime('2/03/2020', '%d/%m/%Y')
for op in OP_LIST:
    if "'" not in op:
        arima = Arima2(op, TODAY)
        pred, val = arima.predict()
        y_hat += pred
        y += val

In [ ]:
errs = []
for i in range(len(y)):
    errs.append(
        np.abs((y[i] - y_hat[i]))
    )
errs.sort()

temp_err = []
for i in errs:
    if i<100:
        temp_err.append(i)

rcParams.update({'figure.autolayout': True})
rcParams.update({'font.size': 12})
f, (ax, ax2) = plt.subplots(2, 1, gridspec_kw = {'height_ratios':[1, 4]}, sharex=True)

ax2.hist(errs, 40, rwidth=.7, color='steelblue')
ax.hist(errs, 40, rwidth=.7, color='steelblue')

ax2.set_ylim(0, 50)
ax.set_ylim(750, 1200)

ax.spines['bottom'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax.xaxis.tick_top()

ax.grid()
ax2.grid()
ax2.set_ylabel('Number of Occurrences')
ax2.set_xlabel('Absolute Point-wise Forecasting Error')
if SAVE:
    plt.savefig('../fig/pointerr.png', transparent=True)

## Genetic: Generations and Profits Distributions

In [ ]:
data = pd.read_csv('../data/predictedOps.csv').drop(columns=['Unnamed: 0']).set_index('op')
gen2 = pd.read_csv('../data/gen2.csv')
gen1 = pd.read_csv('../data/gen1.csv')

rcParams.update({'figure.autolayout': True})
rcParams.update({'font.size': 12})
bins = np.arange(min(gen1['gen1']), max(gen1['gen1']), 30)

plt.hist(gen2['gen2'], bins=bins, color = ['steelblue'], rwidth=.7)

plt.xticks(bins, rotation=90)
plt.ylim(0,22)
plt.xlim(0, 970)
plt.xlabel('Number of Generations')
plt.ylabel('Number of Occurrences')
plt.legend()
if SAVE:
    plt.savefig('../fig/geneticGenerations.png', transparent = True)

In [ ]:
rcParams.update({'figure.autolayout': True})
rcParams.update({'font.size': 12})
rcParams.update({'figure.autolayout': True})
bins = np.arange(min(gen1['prof1']), max(gen1['prof1']), 4000)

#plt.hist(gen2['prof2'], bins=bins, color = 'steelblue', rwidth=.8)

vals = gen2['prof2'].copy().values
vals.sort()
vals = (vals - min(vals))/(max(vals)-min(vals))*100
plt.plot(vals, linewidth=2)
plt.grid(linestyle='-.')
plt.xlabel('Number of Operators')
plt.ylabel('Normalized Profit [%]')
if SAVE:
    plt.savefig('../fig/geneticProfits.png', transparent = True)

# 3) Case Study
## Fitness Trend

In [ ]:
profit = []
genetic = Genetic('IREN ENERGIA SPA', data, TODAY, 8.0)
p, _, o = genetic.run()
profit.append(p)

In [ ]:
profit2 = profit[0].copy()
for i in range(len(profit2)):
    profit2[i] = profit2[i]/1000

rcParams.update({'figure.autolayout': True})
rcParams.update({'font.size': 13})
plt.plot(profit2)
plt.fill_between(range(len(profit2)), profit2, alpha=.2)
plt.xlim(0, len(profit2))
plt.ylim(0, 55)
plt.xlabel('Generations')
plt.ylabel('Fitness Value [K\u20ac]')
plt.grid(linestyle='-.')
if SAVE:
    plt.savefig('../fig/irenprofit.png', transparent=True)

## Delta Strategy

In [ ]:
data_f = data.loc['IREN ENERGIA SPA']
sol = _[0][0]
delta = []
delta.append(-data_f['MGPpO']+sol[0])
delta.append(-(data_f['MGPqO']-sol[1])/1000)
delta.append(-data_f['MGPpD']+sol[2])
delta.append(-(data_f['MGPqD']-sol[3])/1000)
delta.append(-data_f['MIpO']+sol[4])
delta.append(-(data_f['MIqO']-sol[5])/1000)
delta.append(-data_f['MIpD']+sol[6])
delta.append(-(data_f['MIqD']-sol[7])/1000)
delta.append(-data_f['MSDpO']+sol[8])
delta.append(-(data_f['MSDqO']-sol[9])/1000)
delta.append(-data_f['MSDpD']+sol[10])
delta.append(-(data_f['MSDqD']-sol[11])/1000)
sol_o = [
    data_f['MGPpO'], data_f['MGPqO'], data_f['MGPpD'], 
    data_f['MGPqD'], data_f['MIpO'], data_f['MIqO'],
    data_f['MIpD'], data_f['MIqD'], data_f['MSDpO'],
    data_f['MSDqO'], data_f['MSDpD'], data_f['MSDqD']
]

rcParams.update({'figure.autolayout': True})
rcParams.update({'font.size': 13})
fig, ax = plt.subplots(figsize=(6.4, 4.8))
x = np.arange(len(delta))

bar_width = 0.4
ax2 = ax.twinx() 

cnt = 0
for i in np.arange(0,len(delta), 2):
    ax.bar(x[i]+cnt, delta[i],
        width=.8, color='steelblue')

    ax2.bar(x[i+1]+cnt, delta[i+1],
        width=.8, color='brown')

    cnt+=2.5
plt.grid(linestyle='-.')    
tick = np.arange(0.5,25,4.5)  
plt.xticks(tick)
ax2.set_ylim(-30, 30)
ax.set_ylim(-270, 270)
ax.set_ylabel('$\Delta$ Bidded Prices [\u20ac/MWh]')
ax2.set_ylabel('$\Delta$ Bidded Quantities [GWh]')
plt.xticks(tick, labels=[
    'MGP\nOFF', 'MGP\nDEM', 'MI\nOFF', 'MI\nDEM', 'MSD\nOFF', 'MSD\nDEM'])    
ax.legend(['Prices'],loc='lower left', bbox_to_anchor= (.0, 1.01), ncol=2,
            borderaxespad=0, frameon=False)
ax2.legend(['Quantities'],loc='lower left', bbox_to_anchor= (.3, 1.01), ncol=2,
            borderaxespad=0, frameon=False)
if SAVE:
    plt.savefig('../fig/casestudy.png', transparent=True)